# Mengidentifikasi genre musik

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import librosa as lb
import os
from PIL import Image
import keras

Using TensorFlow backend.


### pakai librosa untuk mengconvert sound ke float

### Dataset
dataset 10 genre
 1. Blues(100)
 2. Classical(100)
 3. Country(100)
 4. Disco(100)
 5. Hiphop(100)
 6. Jazz(100)
 7. Metal(100)
 8. Pop(100)
 9. Reggae(100)
 10. Rock(100)
total 1000 lagu

### Mengambil data dari setiap musik dan mengubahnya ke spectrograms

In [2]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
allgenre = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

for i in allgenre:
    pathlib.Path(f'datasimg/{i}').mkdir(parents=True, exist_ok=True)   
    
    for filename in os.listdir(f'./allgenre/{i}'):
        songname = f'./allgenre/{i}/{filename}'
        y, sr = lb.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'datasimg/{i}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

F:\ANACONDAA\lib\site-packages\matplotlib\axes\_axes.py:7739: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


<Figure size 720x720 with 0 Axes>

### meng-extract data features dari spectogram
#####https://medium.com/heuristics/audio-signal-feature-extraction-and-clustering-935319d2225
* Mel-frequency cepstral coefficients
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [3]:
featurename = 'filename  chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

for i in range(1, 21):
    featurename += f' mfcc{i}'
    
featurename += ' label'
featurename = featurename.split()

### Menulis data-data file baru

In [4]:
file = open('data.csv', 'w', newline='')
rmse = lb.feature.rms(y=y)

with file:
    writer = csv.writer(file)
    writer.writerow(featurename)
    
allgenre = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

for i in allgenre:
    
    for filename in os.listdir(f'./allgenre/{i}'):
        songname = f'./allgenre/{i}/{filename}'
        y, sr = lb.load(songname, mono=True, duration=30)
        
        chroma_stft = lb.feature.chroma_stft(y=y, sr=sr)
        spec_cent = lb.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = lb.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = lb.feature.spectral_rolloff(y=y, sr=sr)
        zcr = lb.feature.zero_crossing_rate(y)
        mfcc = lb.feature.mfcc(y=y, sr=sr)
        
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'  
        
        for e in mfcc:
            to_append += f' {np.mean(e)}'
            
        to_append += f' {i}'
        file = open('data.csv', 'a', newline='')
        
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

### meng-analisa data-data mengunakan pandas

In [5]:
data = pd.read_csv('data.csv')
data.head()
data.shape

(1000, 28)

In [6]:
# menghapus kolom number
data = data.drop(['filename'],axis=1)

### mengubah hasil output

In [7]:
listgenre = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(listgenre)

### melakukan scaling terhadap ukuran feature

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

### Membagi data menjadi training dan testing

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(y_train)
len(y_test)
X_train[10]

array([-0.84250827,  0.        , -1.38042918, -1.30633306, -1.45978919,
       -1.18211907, -0.26731443,  1.94622997, -0.40956695, -0.34677642,
        0.70916172, -1.93403168, -0.05435653, -0.9546355 , -0.22193281,
       -1.3086826 ,  0.9868096 , -1.22953889, -0.27597007,  0.40068073,
       -0.31980677, -0.92260604, -0.50221837, -0.6327032 ,  0.39715946,
       -0.08867692])

### klasifikasi menggunakan keras

In [10]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(X_train,y_train,epochs=20,batch_size=128)    

Epoch 1/20
800/800 [==============================] - 0s 303us/step - loss: 2.1909 - accuracy: 0.2450
Epoch 2/20
800/800 [==============================] - 0s 17us/step - loss: 1.8816 - accuracy: 0.3675
Epoch 3/20
800/800 [==============================] - 0s 15us/step - loss: 1.6514 - accuracy: 0.4162
Epoch 4/20
800/800 [==============================] - 0s 16us/step - loss: 1.4648 - accuracy: 0.4800
Epoch 5/20
800/800 [==============================] - 0s 15us/step - loss: 1.3283 - accuracy: 0.5562
Epoch 6/20
800/800 [==============================] - 0s 15us/step - loss: 1.2294 - accuracy: 0.5713
Epoch 7/20
800/800 [==============================] - 0s 15us/step - loss: 1.1449 - accuracy: 0.5962
Epoch 8/20
800/800 [==============================] - 0s 15us/step - loss: 1.0623 - accuracy: 0.6488
Epoch 9/20
800/800 [==============================] - 0s 16us/step - loss: 1.0113 - accuracy: 0.6650
Epoch 10/20
800/800 [==============================] - 0s 16us/step - loss: 0.9635 - accur

In [13]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 329us/step


In [14]:
print('test_acc: ',test_acc)

test_acc:  0.6449999809265137


In [15]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

### TRAIN TRAIN TRAIN TRAINNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [16]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=100,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)
results

Train on 600 samples, validate on 200 samples
Epoch 1/100
600/600 [==============================] - 0s 566us/step - loss: 2.3126 - accuracy: 0.0817 - val_loss: 2.1984 - val_accuracy: 0.2700
Epoch 2/100
600/600 [==============================] - 0s 22us/step - loss: 2.1603 - accuracy: 0.3033 - val_loss: 2.0754 - val_accuracy: 0.3250
Epoch 3/100
600/600 [==============================] - 0s 23us/step - loss: 2.0282 - accuracy: 0.3233 - val_loss: 1.9542 - val_accuracy: 0.3500
Epoch 4/100
600/600 [==============================] - 0s 20us/step - loss: 1.8924 - accuracy: 0.3500 - val_loss: 1.8576 - val_accuracy: 0.3950
Epoch 5/100
600/600 [==============================] - 0s 20us/step - loss: 1.7786 - accuracy: 0.3800 - val_loss: 1.7665 - val_accuracy: 0.4050
Epoch 6/100
600/600 [==============================] - 0s 20us/step - loss: 1.6731 - accuracy: 0.4117 - val_loss: 1.6834 - val_accuracy: 0.3800
Epoch 7/100
600/600 [==============================] - 0s 20us/step - loss: 1.5734 - accu

[1.630556435585022, 0.6549999713897705]

In [17]:
results

[1.630556435585022, 0.6549999713897705]

## Predict Predict Predict Predictttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt

In [18]:
predictions = model.predict(X_test)

In [19]:
predictions[0].shape

(10,)

In [20]:
np.sum(predictions[0])

1.0000001

In [21]:
np.argmax(predictions[0])

3

In [22]:
###notes
Librosa -> convert sound to float
OS -> create remove move change folders
keras -> libary yang isinya kurang lebih ann
###repppprensi
https://medium.com/heuristics/audio-signal-feature-extraction-and-clustering-935319d2225

SyntaxError: invalid syntax (<ipython-input-22-17193157b4f3>, line 2)